In [6]:
import wikipedia
import pandas as pd

In [2]:
df = pd.read_csv('20July')
df

,Unnamed: 0,index,course_number,transcripts,length,token_transcripts,key_words,key_words_from_original,key_words_from_token
0,0,0,630,Hi and welcome to the Master of Applied Data S...,3079,Master Applied Course Causal Inference Alain C...,['causality'],causality,causal nature
1,1,1,630,"All right. So, I got a question in the forum f...",9387,right question forum RedFake programming assig...,['discrimination'],discrimination,female male indicative discrimination
2,2,2,630,"Hi, Daniel. Hello, Cohan. Hi. How are you? Ver...",20115,Daniel Hello Cohan good bad complain complain ...,['samples'],samples,daniel hey daniel
3,3,3,630,"the red line. Now, next time we draw another 5...",20116,red line time draw 50 people overshooting draw...,['expectation'],expectation,average sample
4,4,4,630,"In this video, we're going to take a closer lo...",16163,video closer problem selection bias Selection ...,['universal healthcare'],universal healthcare,health health insurance selection bias
...,...,...,...,...,...,...,...,...,...
1204,1204,1226,680,"In this lecture, we're talking specifically ab...",26960,lecture talking specifically dashboard designi...,['student dashboards'],student dashboards,able interpret dashboard tell student informat...
1205,1205,1227,680,"In this lecture, we're going to be talking abo...",12210,lecture talking data-driven intervention speci...,['learning analytics'],learning analytics,help teacher use assessment result student cla...
1206,1206,1228,680,"Record. Alright, looks like we're recording on...",20241,Record Alright look recording computer upload ...,['assignments'],assignments,impactful student learning result
1207,1207,1229,680,n math and so the idea is if you were in two d...,20242,n math idea different state United States cros...,['common assessments'],common assessments,analysis lot charter school


In [1]:
# A basic function that works
def find_url(keyword):
    try:
        page = wikipedia.page(keyword).url
        
    except wikipedia.exceptions.DisambiguationError as e:
        page = 'Disambiguation'

    except wikipedia.exceptions.PageError: 
        page = None
        
    return page

In [5]:
# Apply the find_url function to "key_words_from_original" column
df['url'] = df['key_words_from_original'].apply(find_url)

/Users/zhouwei/opt/anaconda3/lib/python3.8/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/zhouwei/opt/anaconda3/lib/python3.8/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [6]:
df.url.nunique() #More than 50% of origianl videos have a URL by applying the basic find_url function

624

In [7]:
# Filter the dataset and get a new dataset with disambiguation errors when executing find_url function
df_disambiguation = df[df.url=='Disambiguation']

In [11]:
import spacy
nlp=spacy.load('en_core_web_sm')

In [12]:
import spacy
def cosine_similarity(doc1,doc2): 
    nlp1 = nlp(doc1)
    nlp2 = nlp(doc2)
    similarity = nlp1.similarity(nlp2)
    return similarity

# Solve Disambiguation Issues

In [2]:
# This function doesn't work well. We won't this use function to generate URL pages, instead, we'll create options for each 
# data instance, and then create URLs. Please see below: Toby's Solution. 
def disambiguation_url(keyword):
    try:
        page = wikipedia.page(keyword).url
       
    except wikipedia.exceptions.DisambiguationError as e:
        #print(type(e.options)) #e.options is a list
        temp=[] 
        for option in e.options: 
            temp.append(cosine_similarity(wikipedia.summary(option,sentences=1),keyword))
        page = wikipedia.page(temp.index(max(temp))).url
        print(page)
    return page

In [ ]:
df_disambiguation['url'] = df_disambiguation['key_words_from_original'].apply(disambiguation_url)

In [ ]:
df.to_csv('df_URLs')

# Solution - Solved!

In [196]:
# STEP1: All keywords input will get a list of options output
def get_options(keyword):
    options = list(keyword)
    try:
        page = wikipedia.page(keyword).url
    except wikipedia.exceptions.DisambiguationError as e:
        options = e.options
    except wikipedia.PageError as e:
        options = wikipedia.search(keyword)
    return options

In [18]:
# STEP2: Calculate the similarity between original keyword and list of options output
def cosine_similarity(doc1,doc2): 
    nlp1 = nlp(doc1)
    nlp2 = nlp(doc2)
    similarity = nlp1.similarity(nlp2)
    return similarity

In [5]:
# STEP3: Create a function to evaluate option in options 
def select_option(options,keyword): #options is a list

    if len(options)==1:
        option = options[0] # The only result will be used as searching query
        
    elif len(options)>1: 
        try: 
            temp=[]
            for option in options: 
                score=cosine_similarity(wikipedia.summary(option,sentences=1),keyword)
                temp.append(score)
            option = options[temp.index(max(temp))]
            
        except:
            option = None
    
    return option

In [199]:
get_options('machine learning')

['Machine learning',
 'Active learning (machine learning)',
 'Hyperparameter (machine learning)',
 'Quantum machine learning',
 'Boosting (machine learning)',
 'Adversarial machine learning',
 'Attention (machine learning)',
 'Automated machine learning',
 'Deep learning',
 'Feature (machine learning)']

In [8]:
disambiguation_url('pandas')

'https://en.wikipedia.org/wiki/Giant_panda'

In [25]:
df.to_csv('df_options')

In [226]:
df_options = pd.read_csv('df_options')
df_options['options'][13][1:-1].split(',')

["'Regression discontinuity design'",
 " 'David Lee (economist)'",
 " 'Wavelet'",
 " 'Joshua Angrist'",
 " 'John von Neumann'",
 " 'Economic democracy'",
 " 'Franciscus Patricius'"]

In [285]:
df_options['option'] = df_options.apply(lambda x: select_option(x.options[1:-1].split(','),x.key_words_from_original),axis=1)


/var/folders/yr/xlyqmz0x0s74vjlx_3828vzr0000gn/T/ipykernel_10133/3264575333.py:5: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity = nlp1.similarity(nlp2)
/Users/zhouwei/opt/anaconda3/envs/nlp_env/lib/python3.8/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that cause

In [286]:
df_options.to_csv('df_2Aug_updated')

In [287]:
df=pd.read_csv('df_2Aug_updated')

In [292]:
df['url'] = df['option'].apply(create_url)

/Users/zhouwei/opt/anaconda3/envs/nlp_env/lib/python3.8/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/zhouwei/opt/anaconda3/envs/nlp_env/lib/python3.8/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [301]:
df.to_csv('2Aug_URL')

In [312]:
df[df['url'].notnull()]

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,index,course_number,transcripts,length,token_transcripts,key_words,key_words_from_original,key_words_from_token,options,option,url
0,0,0,0,0,630,Hi and welcome to the Master of Applied Data S...,3079,Master Applied Course Causal Inference Alain C...,['causality'],causality,causal nature,['causality'],'causality',https://en.wikipedia.org/wiki/Causality
1,1,1,1,1,630,"All right. So, I got a question in the forum f...",9387,right question forum RedFake programming assig...,['discrimination'],discrimination,female male indicative discrimination,['discrimination'],'discrimination',https://en.wikipedia.org/wiki/Discrimination
4,4,4,4,4,630,"In this video, we're going to take a closer lo...",16163,video closer problem selection bias Selection ...,['universal healthcare'],universal healthcare,health health insurance selection bias,['universal healthcare'],'universal healthcare',https://en.wikipedia.org/wiki/Universal_health...
5,5,5,5,5,630,"In this first video, we're going to talk about...",11298,video important causality challenging identify...,['causation'],causation,useful correlations hint causation example reason,"['Causality', 'Causality (physics)', 'Causatio...",'Causation (law)',https://en.wikipedia.org/wiki/Causation_(law)
6,6,6,6,6,630,This module is about randomized experiments. R...,11229,module randomized experiment Randomized experi...,['average treatment effects'],average treatment effects,average treatment control group column,['average treatment effects'],'average treatment effects',https://en.wikipedia.org/wiki/Average_treatmen...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1203,1203,1203,1203,1225,680,Today we're speaking with Dr. Rebecca Ferguson...,10935,Today speaking Dr. Rebecca Ferguson senior lec...,['disabled students learning'],disabled students learning,accessibility student disability focus,['disabled students learning'],'disabled students learning',https://en.wikipedia.org/wiki/Learning_disability
1204,1204,1204,1204,1226,680,"In this lecture, we're talking specifically ab...",26960,lecture talking specifically dashboard designi...,['student dashboards'],student dashboards,able interpret dashboard tell student informat...,['student dashboards'],'student dashboards',https://en.wikipedia.org/wiki/List_of_United_S...
1205,1205,1205,1205,1227,680,"In this lecture, we're going to be talking abo...",12210,lecture talking data-driven intervention speci...,['learning analytics'],learning analytics,help teacher use assessment result student cla...,['learning analytics'],'learning analytics',https://en.wikipedia.org/wiki/Learning_analytics
1207,1207,1207,1207,1229,680,n math and so the idea is if you were in two d...,20242,n math idea different state United States cros...,['common assessments'],common assessments,analysis lot charter school,['common assessments'],'common assessments',https://en.wikipedia.org/wiki/Formative_assess...


In [12]:
df = pd.read_csv('2Aug_URL')

In [15]:
survey_df = df[['key_words_from_original','url']]

In [29]:
survey_df=survey_df.dropna()

In [42]:
survey_df.to_csv(r'survey.txt', header=None, index=None, sep=' ', mode='a')

In [57]:
import re

In [75]:
with open('survey.txt','r') as file: 
    i = 1
    for line in file.readlines(): 
        new_line = re.sub('\shttps',', URL is: https',line)
        new_line = str(i)+'.'+ 'Key Word is:'+ new_line
        new_line = new_line + '\n' + "choice1: relevant" + '\n' + "choice2: Non relevant" + '\n'
        i += 1
        print(new_line)

1.Key Word is:causality, URL is: https://en.wikipedia.org/wiki/Causality

choice1: relevant
choice2: Non relevant

2.Key Word is:discrimination, URL is: https://en.wikipedia.org/wiki/Discrimination

choice1: relevant
choice2: Non relevant

3.Key Word is:"universal healthcare", URL is: https://en.wikipedia.org/wiki/Universal_health_care

choice1: relevant
choice2: Non relevant

4.Key Word is:causation, URL is: https://en.wikipedia.org/wiki/Causation_(law)

choice1: relevant
choice2: Non relevant

5.Key Word is:"average treatment effects", URL is: https://en.wikipedia.org/wiki/Average_treatment_effect

choice1: relevant
choice2: Non relevant

6.Key Word is:"sample statistics", URL is: https://en.wikipedia.org/wiki/Sampling_(statistics)

choice1: relevant
choice2: Non relevant

7.Key Word is:"new computer", URL is: https://en.wikipedia.org/wiki/Computer

choice1: relevant
choice2: Non relevant

8.Key Word is:"post treatment time series", URL is: https://en.wikipedia.org/wiki/Post-exposure